# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [61]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import collections


# Import API key
from api_keys import geoapify_key

In [62]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,keflavik,64.0049,-22.5624,275.39,60,75,6.17,IS,1703024134
1,1,flying fish cove,-10.4217,105.6791,293.97,94,22,0.00,CX,1703024129
2,2,isafjordur,66.0755,-23.1240,265.34,78,100,3.92,IS,1703024191
3,3,tiksi,71.6872,128.8694,255.36,89,100,4.69,RU,1703024191
4,4,cabo san lucas,22.8909,-109.9124,301.32,50,0,6.17,MX,1703024116


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [63]:
%%capture --no-display

# Configure the map plot
coordinates = {
    "Lat": (city_data_df["Lat"]),
    "Lng": (city_data_df["Lng"]),
    "City": (city_data_df["City"]),
    "Humidity": (city_data_df["Humidity"])
}

# Display the map
coordinates

{'Lat': 0      64.0049
 1     -10.4217
 2      66.0755
 3      71.6872
 4      22.8909
         ...   
 564    32.8995
 565     7.7029
 566     4.5500
 567    28.2364
 568    45.7668
 Name: Lat, Length: 569, dtype: float64,
 'Lng': 0      -22.5624
 1      105.6791
 2      -23.1240
 3      128.8694
 4     -109.9124
          ...   
 564   -105.9603
 565     27.9953
 566     43.0333
 567     33.6254
 568    -61.2985
 Name: Lng, Length: 569, dtype: float64,
 'City': 0              keflavik
 1      flying fish cove
 2            isafjordur
 3                 tiksi
 4        cabo san lucas
              ...       
 564          alamogordo
 565                 wau
 566                yeed
 567              el-tor
 568         princeville
 Name: City, Length: 569, dtype: object,
 'Humidity': 0      60
 1      94
 2      78
 3      89
 4      50
        ..
 564    29
 565    30
 566    43
 567    67
 568    96
 Name: Humidity, Length: 569, dtype: int64}

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [64]:
# Narrow down cities that fit criteria and drop any results with null values
weather = city_data_df.loc[:, "Max Temp"]
                           
# Drop any rows with null values

weather = weather.dropna()

# Display sample data
display(weather)

0      275.39
1      293.97
2      265.34
3      255.36
4      301.32
        ...  
564    292.91
565    299.77
566    301.84
567    287.23
568    286.83
Name: Max Temp, Length: 569, dtype: float64

### Step 3: Create a new DataFrame called `hotel_df`.

In [65]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df.head()

0    275.39
1    293.97
2    265.34
3    255.36
4    301.32
Name: Max Temp, dtype: object

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [66]:
# Set parameters to search for a hotel
radius = 10000
params = {
"apiKey" : geoapify_key

}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for row in hotel_df.itertuples():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude},{radius}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


AttributeError: 'Series' object has no attribute 'itertuples'

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [71]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points()

# Display the map
hotel_map